# RedisTimeseriesManager Example: Market Data(OHLCV)

### Demonstrating a high-performance market price data downsampling mechanism using *redis* backend and `RedisTimeseriesManager`

In this example, we are going to maintain the data of some financial markets. We have chosen the `cryptocurrency` and `irx` for our example. Each market contain several instruments that we refer to them as symbols and we collect OHLCV(`open`, `high`, `low`, `close`, `volume`) data for each symbol.

The raw data is directly collected from the market with the resolution of seconds and we insert them in `raw` timeframe. Then we compress(downsample) the data to timeframes of `1m`, `1h` and `1d`. As the names `open`, `high`, `low`, `close`, `volume` implies, we use the `FIRST` aggregator for `open`, `MAX` for `high`, `MIN` for `low`, `LAST` for `close` and the `SUM` aggregator for `volume` to compress the data and build the appropriate timeframes of data.

We also want to keep `1m` data for just one week, `1h` for one month and respectively `1d` data for a year.
In this Example, we use the classifier 1(c1) to identify the market(here `cryptocurrency` or `irx`) and the classifier 2(c2) for the symbols.

In [1]:
import time, datetime, random
from pytz import timezone

from redis_timeseries_manager import RedisTimeseriesManager

settings = {
    'host': 'localhost',
    'port': 6379,
    'db': 13,
    'password': None,
}

class MarketData(RedisTimeseriesManager):
    _name = 'markets'
    _lines = ['open', 'high', 'low', 'close', 'volume']
    _timeframes = {
        'raw': {'retention_secs': 60*60*24*4}, # retention 4 days
        '1m': {'retention_secs': 60*60*24*7, 'bucket_size_secs': 60}, # retention 7 day; timeframe 60 secs
        '1h': {'retention_secs': 60*60*24*30, 'bucket_size_secs': 60*60}, # retention 1 month; timeframe 3600 secs
        '1d': {'retention_secs': 60*60*24*365, 'bucket_size_secs': 60*60*24}, # retention 1 year; timeframe 86400 secs
    }

    #compaction rules
    def _create_rule(self, c1:str, c2:str, line:str, timeframe_name:str, timeframe_specs:str, source_key:str, dest_key:str):
        if line == 'open':
            aggregation_type = 'first'
        elif line == 'close':
            aggregation_type = 'last'
        elif line == 'high':
            aggregation_type = 'max'
        elif line == 'low':
            aggregation_type = 'min'
        elif line == 'volume':
            aggregation_type = 'sum'
        else:
            return
        bucket_size_secs = timeframe_specs['bucket_size_secs']
        self._set_rule(source_key, dest_key, aggregation_type, bucket_size_secs)
    
    @staticmethod
    def print_data(data):
        for ts, open, high, low, close, volume in data:
            print(f"{datetime.datetime.fromtimestamp(ts, tz=timezone('UTC')):%Y-%m-%d %H:%M:%S}, open: {open}, high: {high}, low: {low}, close: {close}, volume: {volume}")

In [2]:
md = MarketData(**settings)

In this example, we don't create timeseries explicitly using the `create()` method. Instead, the series are created automatically while inserting data by turning on the `create_inplace` option.

In [3]:
crypto_btc = []
crypto_eth = []
irx_usd = []
# generating random data from 2020-01-01 to 2020-03-01; raw(seconds) resolution
for ts in range(1577836800, 1583020800, 60):
    btc = (random.randint(20000, 21000), random.randint(10000000, 20000000))
    eth = (random.randint(1500, 1600), random.randint(1000000, 2000000))
    usd = (random.randint(30000, 35000), random.randint(1000, 2000))
    crypto_btc.append([ts, btc[0], btc[0], btc[0], btc[0], btc[1]])
    crypto_eth.append([ts, eth[0], eth[0], eth[0], eth[0], eth[1]])
    irx_usd.append([ts, usd[0], usd[0], usd[0], usd[0], usd[1]])

# adding data
print(md.add(
    data=crypto_btc,
    c1='crypto',
    c2='btc',
    create_inplace=True,
)[1], 'records inserted for crypto:btc:raw')

print(md.add(
    data=crypto_eth,
    c1='crypto',
    c2='eth',
    create_inplace=True,
)[1], 'records inserted for crypto:eth:raw')

print(md.add(
    data=irx_usd,
    c1='irx',
    c2='usd',
    create_inplace=True,
)[1], 'records inserted for irx:usd:raw')


432000 records inserted for crypto:btc:raw
432000 records inserted for crypto:eth:raw
432000 records inserted for irx:usd:raw


Taking a look at `btc` data in `raw` timeframe, printing the first 10 records. As you can see, data older than retention period is deleted:

In [4]:
data = md.read(
    c1='crypto',
    c2='btc',
    timeframe='raw',
)
md.print_data(data[1][:10])

2020-02-25 23:59:00, open: 20164.0, high: 20164.0, low: 20164.0, close: 20164.0, volume: 16114266.0
2020-02-26 00:00:00, open: 20142.0, high: 20142.0, low: 20142.0, close: 20142.0, volume: 19860362.0
2020-02-26 00:01:00, open: 20444.0, high: 20444.0, low: 20444.0, close: 20444.0, volume: 15117048.0
2020-02-26 00:02:00, open: 20563.0, high: 20563.0, low: 20563.0, close: 20563.0, volume: 15013905.0
2020-02-26 00:03:00, open: 20490.0, high: 20490.0, low: 20490.0, close: 20490.0, volume: 19887161.0
2020-02-26 00:04:00, open: 20287.0, high: 20287.0, low: 20287.0, close: 20287.0, volume: 18758443.0
2020-02-26 00:05:00, open: 20553.0, high: 20553.0, low: 20553.0, close: 20553.0, volume: 19935727.0
2020-02-26 00:06:00, open: 20358.0, high: 20358.0, low: 20358.0, close: 20358.0, volume: 15769518.0
2020-02-26 00:07:00, open: 20505.0, high: 20505.0, low: 20505.0, close: 20505.0, volume: 10443168.0
2020-02-26 00:08:00, open: 20354.0, high: 20354.0, low: 20354.0, close: 20354.0, volume: 10145899.0


Making sure data is properly downsampled for the `1h` timeframe of `btc`. Just looking at last 10 records:

In [5]:
data = md.read_last_n_records(
    c1='crypto',
    c2='btc',
    timeframe='1h',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-29 13:00:00, open: 20008.0, high: 20988.0, low: 20005.0, close: 20273.0, volume: 858328529.0
2020-02-29 14:00:00, open: 20394.0, high: 20973.0, low: 20003.0, close: 20139.0, volume: 914675931.0
2020-02-29 15:00:00, open: 20022.0, high: 20966.0, low: 20009.0, close: 20498.0, volume: 904645455.0
2020-02-29 16:00:00, open: 20938.0, high: 20989.0, low: 20021.0, close: 20785.0, volume: 890538152.0
2020-02-29 17:00:00, open: 20334.0, high: 20966.0, low: 20005.0, close: 20450.0, volume: 876584584.0
2020-02-29 18:00:00, open: 20845.0, high: 20979.0, low: 20011.0, close: 20219.0, volume: 857713946.0
2020-02-29 19:00:00, open: 20776.0, high: 20978.0, low: 20007.0, close: 20604.0, volume: 921944356.0
2020-02-29 20:00:00, open: 20905.0, high: 20995.0, low: 20037.0, close: 20275.0, volume: 886750618.0
2020-02-29 21:00:00, open: 20433.0, high: 20988.0, low: 20004.0, close: 20868.0, volume: 898904867.0
2020-02-29 22:00:00, open: 20035.0, high: 20967.0, low: 20021.0, close: 20945.0, volume: 90

And finally the `1d` timeframe for `btc`:

In [6]:
data = md.read_last_n_records(
    c1='crypto',
    c2='btc',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 20684.0, high: 20998.0, low: 20003.0, close: 20536.0, volume: 21569158000.0
2020-02-20 00:00:00, open: 20123.0, high: 20998.0, low: 20000.0, close: 20323.0, volume: 21640277211.0
2020-02-21 00:00:00, open: 20834.0, high: 21000.0, low: 20000.0, close: 20499.0, volume: 21538093898.0
2020-02-22 00:00:00, open: 20840.0, high: 21000.0, low: 20000.0, close: 20018.0, volume: 21535570820.0
2020-02-23 00:00:00, open: 20575.0, high: 21000.0, low: 20001.0, close: 20629.0, volume: 21511962155.0
2020-02-24 00:00:00, open: 20280.0, high: 21000.0, low: 20003.0, close: 20022.0, volume: 21695409203.0
2020-02-25 00:00:00, open: 20239.0, high: 21000.0, low: 20000.0, close: 20164.0, volume: 21644223660.0
2020-02-26 00:00:00, open: 20142.0, high: 21000.0, low: 20001.0, close: 20738.0, volume: 21555308754.0
2020-02-27 00:00:00, open: 20089.0, high: 21000.0, low: 20001.0, close: 20198.0, volume: 21662405165.0
2020-02-28 00:00:00, open: 20764.0, high: 21000.0, low: 20000.0, close: 2

What about Ethereum?

In [7]:
data = md.read_last_n_records(
    c1='crypto',
    c2='eth',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 1596.0, high: 1600.0, low: 1500.0, close: 1556.0, volume: 2143999543.0
2020-02-20 00:00:00, open: 1554.0, high: 1600.0, low: 1500.0, close: 1586.0, volume: 2161415271.0
2020-02-21 00:00:00, open: 1506.0, high: 1600.0, low: 1500.0, close: 1531.0, volume: 2149807932.0
2020-02-22 00:00:00, open: 1579.0, high: 1600.0, low: 1500.0, close: 1578.0, volume: 2154955516.0
2020-02-23 00:00:00, open: 1585.0, high: 1600.0, low: 1500.0, close: 1581.0, volume: 2162673381.0
2020-02-24 00:00:00, open: 1514.0, high: 1600.0, low: 1500.0, close: 1555.0, volume: 2153614376.0
2020-02-25 00:00:00, open: 1566.0, high: 1600.0, low: 1500.0, close: 1507.0, volume: 2155428171.0
2020-02-26 00:00:00, open: 1568.0, high: 1600.0, low: 1500.0, close: 1580.0, volume: 2160880310.0
2020-02-27 00:00:00, open: 1545.0, high: 1600.0, low: 1500.0, close: 1598.0, volume: 2150496148.0
2020-02-28 00:00:00, open: 1576.0, high: 1600.0, low: 1500.0, close: 1540.0, volume: 2168125300.0


We also had a market called `irx`, whats going on with it?

In [8]:
data = md.read_last_n_records(
    c1='irx',
    c2='usd',
    timeframe='1d',
    minimum_timestamp=0,
    n=10,
)
md.print_data(data[2])

2020-02-19 00:00:00, open: 32384.0, high: 35000.0, low: 30000.0, close: 31769.0, volume: 2153204.0
2020-02-20 00:00:00, open: 30362.0, high: 34997.0, low: 30002.0, close: 32887.0, volume: 2165782.0
2020-02-21 00:00:00, open: 34322.0, high: 35000.0, low: 30014.0, close: 32140.0, volume: 2172368.0
2020-02-22 00:00:00, open: 34090.0, high: 34996.0, low: 30006.0, close: 31156.0, volume: 2142013.0
2020-02-23 00:00:00, open: 31106.0, high: 34995.0, low: 30006.0, close: 31497.0, volume: 2155531.0
2020-02-24 00:00:00, open: 33332.0, high: 35000.0, low: 30000.0, close: 32006.0, volume: 2158651.0
2020-02-25 00:00:00, open: 31023.0, high: 35000.0, low: 30008.0, close: 31511.0, volume: 2177522.0
2020-02-26 00:00:00, open: 30033.0, high: 34992.0, low: 30000.0, close: 31871.0, volume: 2147087.0
2020-02-27 00:00:00, open: 34626.0, high: 34998.0, low: 30001.0, close: 30956.0, volume: 2175925.0
2020-02-28 00:00:00, open: 32097.0, high: 34997.0, low: 30004.0, close: 34060.0, volume: 2175474.0


## Other Commands
In the background, several keys are created in the redis db. To inspect the list of keys, run the following command:

In [9]:
md.query_index(return_key_names=True)[1]

['markets:crypto:btc:1d:close',
 'markets:crypto:btc:1d:high',
 'markets:crypto:btc:1d:low',
 'markets:crypto:btc:1d:open',
 'markets:crypto:btc:1d:volume',
 'markets:crypto:btc:1h:close',
 'markets:crypto:btc:1h:high',
 'markets:crypto:btc:1h:low',
 'markets:crypto:btc:1h:open',
 'markets:crypto:btc:1h:volume',
 'markets:crypto:btc:1m:close',
 'markets:crypto:btc:1m:high',
 'markets:crypto:btc:1m:low',
 'markets:crypto:btc:1m:open',
 'markets:crypto:btc:1m:volume',
 'markets:crypto:btc:raw:close',
 'markets:crypto:btc:raw:high',
 'markets:crypto:btc:raw:low',
 'markets:crypto:btc:raw:open',
 'markets:crypto:btc:raw:volume',
 'markets:crypto:eth:1d:close',
 'markets:crypto:eth:1d:high',
 'markets:crypto:eth:1d:low',
 'markets:crypto:eth:1d:open',
 'markets:crypto:eth:1d:volume',
 'markets:crypto:eth:1h:close',
 'markets:crypto:eth:1h:high',
 'markets:crypto:eth:1h:low',
 'markets:crypto:eth:1h:open',
 'markets:crypto:eth:1h:volume',
 'markets:crypto:eth:1m:close',
 'markets:crypto:eth:

You can inspect the info of each key by running the `stats` command as shown in the next cell.

> Note that each key have lablels properly filled with respective data(`c1`, `c2`, `...`), so you can take advantage of *redis multi-timeseries commands* like [TS.MRANGE](https://redis.io/commands/ts.mrange/)


In [10]:
md.stats('crypto', 'btc', 'raw', 'close').__dict__

{'rules': [[b'markets:crypto:btc:1m:close', 60000, b'LAST'],
  [b'markets:crypto:btc:1h:close', 3600000, b'LAST'],
  [b'markets:crypto:btc:1d:close', 86400000, b'LAST']],
 'source_key': None,
 'chunk_count': 4,
 'memory_usage': 17013,
 'total_samples': 5761,
 'labels': {'tl': 'markets',
  'c1': 'crypto',
  'c2': 'btc',
  'line': 'close',
  'timeframe': 'raw'},
 'retention_msecs': 345600000,
 'lastTimeStamp': 1583020740000,
 'first_time_stamp': 1582675140000,
 'chunk_size': 4096,
 'duplicate_policy': 'last'}